In [ ]:
# from new_preprocessing import  DataExtractor, DataPreprocessing



# directory = r"C:\Users\maxda\Studium\Semester6\Anwendungsprojekt\hka-aqm-data"
# new_directory = r"C:\Users\maxda\Studium\Semester6\Anwendungsprojekt\test_new_preprocessing"
# de = DataExtractor(first_directory=directory, new_directory=new_directory)

# de.extract_zip_files(directory, new_directory)
# de.delete_zip_files(directory)

# df = de.get_data(new_directory)

# dp = DataPreprocessing()

# df_new = dp.preprocess_df(df)

In [ ]:
# df_m = df_new[df_new.building_name == "m"]

In [ ]:
# df_m.to_parquet("Preprocessed_M_Data.parquet")

In [ ]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
df_m = pd.read_parquet("../Gebäude_M_Analysis/Preprocessed_M_Data.parquet")

In [ ]:
df_m.columns



Attributsbeschreibung:

    date_time := Datum (Format: YYYY-MM-DD HH:MM:SS)
    device_id := CO2-Ampel-ID
    tmp := Durchschnittliche Temperatur
    hum := Durchschnittliche Luftfeuchtigkeit
    CO2 := Durchschnittlicher CO2-Wert
    VOC := Durchschnittliche Schadstoffbelastung
    vis := Durchschnittlicher Lichtwert
    IR := Durchschnittlicher Infrarotwert
    WIFI := Durchschnittliche Anzahl an WiFi-Geräten
    BLE := Durchschnittliche Anzahl an Bluetoothgeräten
    rssi := received signal strength indication, Empfangsfeldstärke kabelloser Kommunikationsanwendungen
    channel_rssi := Gesamtstärke des ganzen Empfangs
    snr := Signal-Rausch-Verhältnis
    gateway := Genutztes Gateway für die Übertragung des Datenpunktes
    channel_index :=
    spreading_factor :=
    bandwidth :=
    f_cnt :=
    building_name := Name des Gebäudes, in dem sich die CO2-Ampel befindet



Verteilung Messungen im Verlauf der Zeit

In [ ]:
df_m.dtypes

In [ ]:
df_m["date"] = df_m["date_time"].dt.date
daily_counts = df_m.groupby('date').size().reset_index(name='count')

px.line(daily_counts, x="date", y="count")

Verteilung Messwerte pro Raum im Verlauf der Zeit

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

m_building_rooms = df_m.room_number.unique()

num_rooms = len(m_building_rooms)

cols = 4

# Anzahl der Zeilen berechnen
rows = - (- num_rooms // cols)  # ceiling division

# Subplots erstellen
fig = make_subplots(rows=rows, cols=cols, shared_xaxes=True, shared_yaxes=True, subplot_titles=[f"Room {room}" for room in m_building_rooms])

# Schleife über jeden Raum
for i, room in enumerate(m_building_rooms, start=1):
    df_room = df_m[df_m.room_number == room]
    daily_count_room = df_room.groupby('date').size().reset_index(name='count')
    
    # Linie hinzufügen
    row = (i - 1) // cols + 1
    col = (i - 1) % cols + 1
    fig.add_trace(go.Scatter(x=daily_count_room['date'], y=daily_count_room['count'], mode='lines',line=dict(color='blue'), name=f"Room {room}"), row=row, col=col)

# Layout anpassen
fig.update_layout(height=300*rows, width=1800, title_text="Daily Measurement Counts per Room", showlegend=False)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Count")

# Anzeige
fig.show()





In [ ]:
df_m.columns

CO2 - Werte Tagesüberblick

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Berechnen der täglichen Mittelwerte für CO2 und VOC
CO2_daily = df_m.groupby('hour', as_index=False).agg({"CO2":"mean", "VOC":"mean"})

# Erstellen eines Subplots mit 1 Zeile und 1 Spalte und zwei y-Achsen
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Hinzufügen der CO2-Linie
fig.add_trace(
    go.Scatter(x=CO2_daily["hour"], y=CO2_daily["CO2"], name="CO2"),
    secondary_y=False,
)

# Hinzufügen der VOC-Linie
fig.add_trace(
    go.Scatter(x=CO2_daily["hour"], y=CO2_daily["VOC"], name="VOC"),
    secondary_y=True,
)

# Aktualisieren der Layout-Parameter für die Achsen
fig.update_layout(
    title_text="CO2 and VOC Levels by Hour"
)

fig.update_xaxes(title_text="Hour of the Day")

fig.update_yaxes(title_text="CO2 Levels (ppm)", secondary_y=False)
fig.update_yaxes(title_text="VOC Levels (ppb)", secondary_y=True)

# Anzeigen des Plots
fig.show()

CO2 - Werte Tagesüberblick Raumvergleich

In [ ]:
m_building_rooms = df_m.room_number.unique()

num_rooms = len(m_building_rooms)

cols = 4

# Anzahl der Zeilen berechnen
rows = - (- num_rooms // cols)  # ceiling division

# Subplots erstellen
fig = make_subplots(rows=rows, cols=cols, shared_xaxes=True, shared_yaxes=True, subplot_titles=[f"Room {room}" for room in m_building_rooms])

# Schleife über jeden Raum
for i, room in enumerate(m_building_rooms, start=1):
    df_room = df_m[df_m.room_number == room]
    CO2_daily_room = df_room.groupby('hour', as_index=False).agg({"CO2":"mean"})
    
    # Linie hinzufügen
    row = (i - 1) // cols + 1
    col = (i - 1) % cols + 1
    fig.add_trace(go.Scatter(x=CO2_daily_room["hour"], y=CO2_daily_room['CO2'], mode='lines',line=dict(color='blue'), name=f"Room {room}"), row=row, col=col)

# Layout anpassen
fig.update_layout(height=300*rows, width=1800, title_text="CO2 over a day", showlegend=False)
fig.update_xaxes(title_text="Hour")
fig.update_yaxes(title_text="CO2 - avg")

# Anzeige
fig.show()
